In [ ]:
%load_ext autoreload
%autoreload 2

from ble import get_ble_controller
from base_ble import LOG
from cmd_types import CMD
import time
import numpy as np

LOG.propagate = False

In [ ]:
# Get ArtemisBLEController object
ble = get_ble_controller()

# Connect to the Artemis Device
ble.connect()

# Lab Tasks

## 1. Echo

In [ ]:
ble.send_command(CMD.ECHO, "HiHello")
s = ble.receive_string(ble.uuid['RX_STRING'])
print(s)

## 2. Get Time Millis

In [ ]:
ble.send_command(CMD.GET_TIME_MILLIS, "")
s = ble.receive_string(ble.uuid['RX_STRING'])
print(s)

## 3. Notifications

In [ ]:
def notification_handler(sender, data):
    time = int(data[2:])
    LOG.info(f'Time: {time}')

Sleep to let the response come in

In [ ]:
ble.start_notify(ble.uuid['RX_STRING'], notification_handler)
ble.send_command(CMD.GET_TIME_MILLIS, "")
time.sleep(1)
ble.stop_notify(ble.uuid['RX_STRING'])

## 4. Five Temps

In [ ]:
data_buffer = ''
def buffered_notification_handler(sender, data):
    global data_buffer
    data = ble.bytearray_to_string(data)
    data_buffer += data
    if data[-1] == 'E':
        data = data_buffer[1:-2].split('|')
        data_buffer = ''
        for time, temp in zip(data[0::2], data[1::2]):
            time = int(time[2:])
            temp = float(temp[2:])
            LOG.info(f'Time: {time}, Temp: {temp}')

In [ ]:
ble.start_notify(ble.uuid['RX_STRING'], buffered_notification_handler)
ble.send_command(CMD.GET_TEMP_5s, "")
time.sleep(6)
ble.stop_notify(ble.uuid['RX_STRING'])

## 5. Rapid Temps

In [ ]:
ble.start_notify(ble.uuid['RX_STRING'], buffered_notification_handler)
ble.send_command(CMD.GET_TEMP_5s_RAPID, "")
time.sleep(6)
ble.stop_notify(ble.uuid['RX_STRING'])